In [1]:
import matplotlib
#%matplotlib qt  
#%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.calcium import calcium


/tmp/ipykernel_61444/1392121454.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/home/cat/anaconda3/envs/blockcourse/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Fix missing python packages

In [2]:
# Install missing packages



# then rerun the top/first cell again to see if packages are detected



# Get required raw data for our task from dropbox

In [13]:
# go to google drive download the files from the binarization folder
# make sure you know the location of the .zip file

import webbrowser
url = 'https://drive.google.com/drive/folders/1fiyQdL55S3kkAa0EcUj8fpo50o3zhQcy?usp=sharing'
webbrowser.open(url)



True

In [ ]:
# unzip the binarization_data.zip file 
# you should be able to doubleclick on the file and unzip it using windows zip


# Run [Ca] data binarization algorithms

In [14]:
# set input where our unzipped files are 
# this directory should have all the numpy files outputed from suite2p:
#  e.g.  F.npy
#        Fnew.npy 
#        spks..npy ...
# 

data_dir = '/home/cat/Downloads/binarization_data'
#data_dir = r'C:\...'



In [15]:
# initialize calcium object and load suite2p data
c = calcium.Calcium()                     # initialize a calcium object that will hold our data and process it
c.verbose = True                          # outputs additional information during processing: True / False
c.recompute_binarization = True           # recomputes binarization and other processing steps; False: loads from previous saved locations
c.data_dir = data_dir


In [16]:
# we load the output from suite2p 
c.load_suite2p()                          


  Fluorescence data loading information
         sample rate:  30 hz
         self.F (fluorescence):  (1232, 55740)
         self.Fneu (neuropile):  (1232, 55740)
         self.iscell (Suite2p cell classifier output):  (2131, 2)
              of which number of good cells:  (1232,)
         self.spks (deconnvoved spikes):  (1232, 55740)
         self.stat (footprints structure):  (1232,)
         mean std over all cells :  21.5


In [ ]:
#############################################################
#############################################################
#############################################################
# set flags to save matlab and python data
c.save_python = True         # save output as .npz file 
c.save_matlab = False         # save output as .mat file


In [ ]:
###############################################
##### PARAMETERS FOR RUNNING BINARIZATION #####
###############################################
c.min_width_event_onphase = c.sample_rate//2 # set minimum withd of an onphase event; default: 0.5 seconds
c.min_width_event_upphase = c.sample_rate//4 # set minimum width of upphase event; default: 0.25 seconds


In [ ]:
############# PARAMTERS TO TWEAK ##############
#     1. Cutoff for calling somthing a spike:
#        This is stored in: std_Fluorescence_onphase/uppohase: defaults: 1.5
#                                        higher -> less events; lower -> more events
#                                        start at default and increase if data is very noisy and getting too many noise-events
c.min_thresh_std_onphase = 2.5      # set the minimum thrshold for onphase detection; defatul 2.5
c.min_thresh_std_upphase = 2.5      # set the minimum thershold for uppohase detection; default: 2.5



In [ ]:
#     2. Filter of [Ca] data which smooths the data significantly more and decreases number of binarzied events within a multi-second [Ca] event
#        This is stored in high_cutoff: default 0.5 to 1.0
#        The lower we set it the smoother our [Ca] traces and less "choppy" the binarized traces (but we loose some temporal precision)
c.high_cutoff = 0.5              


In [ ]:

#     3. Removing bleaching and drift artifacts using polynomial fits
#        This is stored in detrend_model_order
c.detrend_model_order = 1 # 1-5 polynomial fit



In [ ]:

################################################
########### RUN BINARIZATION STEP ##############
################################################
# 
c.binarize_fluorescence()

# Visualize results - single cells

In [20]:
###############################################################
######### LOAD BINARZIED DATA AND VISUALIZE CELL ACTIVITY #####
###############################################################
# 
fname = os.path.join(data_dir,'suite2p','plane0','binarized_traces.npz')


In [ ]:
###############################################################
######### LOAD BINARZIED DATA AND VISUALIZE CELL ACTIVITY #####
###############################################################
c = calcium.Calcium()   # we "instantiate" or make a calcium object
c.fname = fname         # we provide the filename of our previously computed file.
c.data_dir = data_dir
c.recompute_binarization = False
c.load_binarization()


In [ ]:
###############################################################
################ SELECT CELL AND VISUALIZE ####################
###############################################################
# pick a random cell to visuzlie
cell_id = 200

# scale value for matching height of 
scale = 100

# 
c.plot_cell_binarization(cell_id, scale)


# Visualize results: rasters of all cells

In [16]:
######################################################
############# SHOW RASTERS ###########################
######################################################
fname = os.path.join(data_dir,'suite2p','plane0','binarized_traces.npz')
#data_dir = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/'

c = calcium.Calcium()
c.data_dir = data_dir
c.fname = fname
c.recompute_binarization = False
c.load_binarization()

c.show_rasters()
